In [13]:
import pandas as pd
from datetime import date

In [19]:
df=pd.read_csv("files/20220228",delimiter="|",header=None,skip_blank_lines=True)
df.columns='a b c'.split()
df.dropna(inplace=True)
df['d']=df['c'].apply(lambda data: json.loads(data)['totalBuyQty'])
df['e']=df['c'].apply(lambda data: json.loads(data)['totalSellQty'])
df2=df.groupby("b").mean()
df2['f']=df2.d/df2.e
# df2.sort_values('f',ascending=False).head()
df2[df2['d']>1000].sort_values('f',ascending=False)

,d,e,f
b,,,
BNL,5827.500000,0.000000,inf
NLO,150488.750000,0.000000,inf
CMF1,26387.500000,150.000000,175.916667
KEF,204324.000000,7162.500000,28.526911
SBI,26638.375000,1856.500000,14.348707
...,...,...,...
SPDL,13979.750000,59825.125000,0.233677
PMHPL,4124.625000,18909.875000,0.218120
CHL,3789.875000,19659.500000,0.192776


In [8]:
df2[df2['d']>1000]

,d,e,f
b,,,
BNL,3820,0,inf
BOKL,1357,255,5.321569
CMF2,30100,100,301.000000
GIMES1,34600,107,323.364486
HDL,3878,576,6.732639
KEF,75724,100,757.240000
LUK,28567,200,142.835000
MMF1,10639,10050,1.058607
NABIL,2021,319,6.335423
